<a href="https://colab.research.google.com/github/SofiaRguez1020/Topicos_Industria_I/blob/main/TI_1_Practica3_Sofia_Rodriguez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3

**Nombre:** Sofia Daniela Rodriguez Saenz

**e-mail:** sofia.rodriguez5540@alumnos.udg.mx

## MODULES

In [65]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import cauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## CLASSES

In [15]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## FUNCTIONS

In [16]:
###############################################################################################
# Brownian Motion
# This function generates a BM walk
###############################################################################################
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    n_steps: number of steps the Brownian Trajectory will take -> int
    speed: speed of the trajectory or step size -> int
    s_pos: initial position -> [x,y] list
  Returns:
    BM_2d_df: DataFrame with x,y points of the full trajectory
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [17]:
###############################################################################################
# Correlated Random Walk
# This function generates a CRW Walk
###############################################################################################
def crw(n_steps=1000, speed=5, s_pos=[0,0], CRW_exponent = 0.5):
  """
  Arguments:
    n_steps: number of steps the Brownian Trajectory will take -> int
    speed: speed of the trajectory or step size -> int
    s_pos: initial position -> [x,y] list
  Returns:
    CRW_df: DataFrame with x,y points of the full trajectory
  """
  #Init DFs
  CRW_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
  CRW_df = pd.concat([CRW_df, temp_df], ignore_index=True)

  # Init velocity vector
  velocity = Vec2d(speed,0)

  for i in range(n_steps-1):
    # Select turn angle
    turn_angle = wrapcauchy.rvs(c=CRW_exponent)
    #
    velocity = velocity.rotated(turn_angle)
    #Update position
    temp_df = pd.DataFrame([{'x_pos': CRW_df.x_pos[i]+velocity.x, 'y_pos': CRW_df.y_pos[i]+velocity.y}])
    CRW_df = pd.concat ([CRW_df,temp_df], ignore_index=True)
  return CRW_df

In [18]:
#######################################################################################################
# Euclidean distance
# This function estimates euclidean distance between two points
#######################################################################################################
def euclidean_distance(q, p):
  a2 = (q[0] - p[0])**2
  b2 = (q[1] - p[1])**2
  c = math.sqrt(a2+b2)
  return c

#######################################################################################################
# Path length calculation
# This function estimates the total path length based on euclidean distance between consecutive steps
#######################################################################################################
def path_length(trajectory):
  """
  Arguments:
    trajectory: Data set containing complete trajectory
  Returns:
    distance: calculated euclidean distance between 2 points
  """
  dis_aux = []
  for i in range (1,trajectory.shape[0]):
    dis_aux.append(euclidean_distance(trajectory.iloc[i-1], trajectory.iloc[i]))
  return pd.DataFrame(np.cumsum(dis_aux), columns=['PL'])


In [19]:
#######################################################################################################
# Mean Squared Displacement
# This function estimates the MSD
#######################################################################################################
def msd(trajectory, n=50):
  """
  Arguments:
    trajectory: Data set containing complete trajectory
    n: Window size
  Returns:
    msd_df: calculated mean squared
  """
  displacement_vec = np.array([euclidean_distance(trajectory.iloc[i-n], trajectory.iloc[i])**2 for i in range(n, trajectory.shape[0],1)])
  msd_df = pd.DataFrame(np.cumsum(displacement_vec)/(trajectory.shape[0] - n), columns=['MSD'])
  return msd_df


In [20]:
###############################################################################################
# Turning angle
# This function calculates the turning angle between three consecutive positions
###############################################################################################
def turning_angle(pos_a, pos_b, pos_c):
    """
    Arguments:
        pos_a: First position coordinates
        pos_b: Second position coordinates
        pos_c: Third position coordinates
    Returns:
        theta: Turning angle
    """
    vec_ab = np.array([pos_b[0] - pos_a[0], pos_b[1] - pos_a[1]])
    norm_ab = np.linalg.norm(vec_ab)

    vec_bc = np.array([pos_c[0] - pos_b[0], pos_c[1] - pos_b[1]])
    norm_bc = np.linalg.norm(vec_bc)

    dot_p = np.dot(vec_ab, vec_bc)

    # Nota: Evitar division por cero con np.finfo(float).eps
    cos_theta = dot_p / (norm_ab * norm_bc + np.finfo(float).eps)

    # Angle orientation
    cross_p = np.cross(vec_ab, vec_bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1

    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta


In [21]:
print(turning_angle([2,2],[5,6],[7,2]))

-2.0344287356224164


In [66]:
###############################################################################################
# Levy walk
# This function generates a levy Walk
###############################################################################################
def levy_flight(n_steps=1000, speed=5, s_pos=[0,0], CRW_exponent = 0.5, alpha = 1.5, beta = 0):
  """
  Arguments:
    n_steps: number of steps the Brownian Trajectory will take -> int
    speed: speed of the trajectory or step size -> int
    s_pos: initial position -> [x,y] list
  Returns:
    CRW_df: DataFrame with x,y points of the full trajectory
  """
  #Init DFs
  Levy_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
  Levy_df = pd.concat([Levy_df, temp_df], ignore_index=True)

  # Init velocity vector
  velocity = Vec2d(speed,0)

  for i in range(n_steps-1):
    # Select turn angle
    turn_angle = wrapcauchy.rvs(c=CRW_exponent)
    #step size
    step_size = levy_stable.rvs(alpha=alpha, beta=beta)

    velocity = velocity.rotated(turn_angle)
    #Update position
    temp_df = pd.DataFrame([{'x_pos': Levy_df.x_pos[i]+(velocity.x*step_size), 'y_pos': Levy_df.y_pos[i]+(velocity.y*step_size)}])
    Levy_df = pd.concat ([Levy_df,temp_df], ignore_index=True)
  return Levy_df

In [22]:
###############################################################################################
# Turning angles for a given trajectory
# This function calculates the all turning angles between consecutive positions in a trajectory
###############################################################################################

def ta_cal(trajectory):
  aux_ta = np.empty(shape=(0))
  # Iterate over trajectory compute turning angles
  for index, row in trajectory[1:-1].iterrows():
      aux_ta = np.append(aux_ta,turning_angle(trajectory.iloc[index-1],trajectory.iloc[index], trajectory.iloc[index+1]))
  TA_df = pd.DataFrame(aux_ta, columns=['TA'])
  return TA_df


## Actividad 1: Path Length - (BM1 vs BM2 vs CRW) (4 pts)

* Implementar función que genere **Brownian Motions** (BM) utilizando **pandas**.
* Implementar función que genere **Correlated Random Walks** (CRW) utilizando pandas.
* Implementar una función alternativa a las ya disponibles en los distintos modulos de python que calcule los valores de la curva de **path length** de una trayectoria.
* Guardar los valores de la métrica en un Data Frame de **pandas**.
* Visualizar con **plotly**.

In [23]:
# Load existing trajectories to test your implementation
# BM speed = 3
BM_2d_df_3 = pd.read_csv('drive/MyDrive/Files_colab/brownian_3.csv')

# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('drive/MyDrive/Files_colab/brownian_6.csv')

# Load existing trajectories to test your implementation
CRW_2d_df_9 = pd.read_csv('drive/MyDrive/Files_colab/crw_6_9.csv')

In [24]:
# Define your function to compute path length for given trajectory
#  *****************See Function section for fuction implementation *********************

In [25]:
# Get Path length calling the function
PL_BM_3 = path_length(BM_2d_df_3)# Add your code here function(BM_2d_df_3)

# Get Path length calling the function
PL_BM_6 = path_length(BM_2d_df_6)# Add your code here function(BM_2d_df_6)

# Get Path length calling the function
PL_CRW_6 = path_length(CRW_2d_df_9)# Add your code here function(CRW_2d_df_9)

In [26]:
PL_BM_3.head()

,PL
0,3.0
1,6.0
2,9.0
3,12.0
4,15.0


In [27]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# First trace BM1
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(PL_BM_3.PL))+1,
    y = PL_BM_3.PL,
    name = 'path_length_BM_3',
    showlegend = True
))

# First trace BM6
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(PL_BM_6.PL))+1,
    y = PL_BM_6.PL,
    name = 'path_length_BM_6',
    line = dict(width = 5),
    showlegend = True
))

# Third trace CRW
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(PL_CRW_6.PL))+1,
    y = PL_BM_6.PL,
    name = 'path_length_CRW_6',
    showlegend = True
))
fig_path_length.show()

## Actividad 2: Mean Squared Displacement - (Brownian vs CRW) (4 pts)

* Generar una trayectoria tipo **BM** y una **CRW**.
* Implementar una función que calcule los valores de la curva de **mean squared displacement** de una trayectoria.
* Guardar metricas en Pandas Data Frame.
* Visualizar con **plotly**.

In [28]:
# Load existing trajectories to test your implementation
# BM speed = 6
BM_2d_df_6 = pd.read_csv('drive/MyDrive/Files_colab/brownian_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('drive/MyDrive/Files_colab/crw_6_9.csv')

In [29]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [30]:
# Define your function to compute Mean Squared Displacement for given trajectory

#  ***************** See Function section for fuction implementation *********************

In [31]:
# Get Mean Squared Displacement calling the function
MSD_BM = msd(BM_2d_df_6, 500)# Add your code here function(BM_2d_df_6)

# Get Mean Squared Displacement calling the function
MSD_CRW = msd(CRW_2d_df_9, 500)# Add your code here function(CRW_2d_df_9)

In [32]:
MSD_BM.head()

,MSD
0,44.943471
1,84.729914
2,130.486832
3,174.947588
4,219.198777


In [33]:
# Init figure
fig_MSD = go.Figure()

# first trace MSD_BM
fig_MSD.add_trace(go.Scatter(
                 x = MSD_BM.index,
                 y = MSD_BM.MSD,
                 name = 'MSD BW 6',
                 showlegend = True
                 ))


# Second trace MSD_CRW
fig_MSD.add_trace(go.Scatter(
                 x = MSD_CRW.index,
                 y = MSD_CRW.MSD,
                 line = dict(width = 4),
                 name = 'MSD CRW 6',
                 showlegend = True
                 ))

fig_MSD.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Generar **CRWs** con dos exponentes diferentes
* Guardar trayectorias en **Pandas** Data Frame
* Implementar una función que calcule los valores de **turning angle** de una trayectoria.
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [52]:
# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.6]
CRW_2d_df_6 = pd.read_csv('drive/MyDrive/Files_colab/crw_6_6.csv')

# Load existing trajectories to test your implementation
# CRW speed = 6,
# wrapcauchy [c = 0.9]
CRW_2d_df_9 = pd.read_csv('drive/MyDrive/Files_colab/crw_6_9.csv')

In [35]:
# Define your function to compute Turning Angles for given trajectory

#  ***************** See Function section for fuction implementation *********************

In [53]:
# Get Turning Angles calling the function
ta_CRW_2d_df_6 = ta_cal(CRW_2d_df_6)# Add your code here function(CRW_2d_df_6)

# Get Turning Angles calling the function
ta_CRW_2d_df_9 = ta_cal(CRW_2d_df_9)# Add your code here function(CRW_2d_df_9)

In [47]:
ta_CRW_2d_df_6.head(10)

,TA
0,0.745668
1,0.181442
2,0.701163
3,0.577802
4,0.413425
5,0.497928
6,0.481365
7,0.425163
8,0.283385
9,0.023399


In [48]:
ta_CRW_2d_df_9.head(10)

,TA
0,0.524886
1,0.484574
2,0.554076
3,0.489555
4,0.687473
5,0.506798
6,0.416439
7,0.497914
8,0.501247
9,0.497042


In [64]:
# Check documentation
# https://plotly.com/python/histograms/

# PLot histogram
fig_met_df_3 = go.Figure()

# Histogram turning angle CRW_2d_df_6
fig_met_df_3.add_trace(go.Histogram(
                 x=ta_CRW_2d_df_6.TA,
                 name = 'observed 0.6',
                 showlegend = True,
                 xbins=dict(size=0.03)
                 ))


# Histogram turning angle CRW_2d_df_9
fig_met_df_3.add_trace(go.Histogram(
                 x=ta_CRW_2d_df_9.TA,
                 name = 'observed 0.9',
                 showlegend = True,
                 xbins=dict(size=0.03)
                 ))


# Add origin distributions
aux_domain = np.linspace(-3, 3, 1000)

out = cauchy.fit(ta_CRW_2d_df_6.TA)
y1 = cauchy.pdf(aux_domain, *out)

fig_met_df_3.add_trace(go.Scatter(
                 x=aux_domain,
                 y= y1*35,
                 name = 'cauchy_0.6',
                 showlegend = True,
                 line=dict(color='blue')
                 ))

out = cauchy.fit(ta_CRW_2d_df_9.TA)
y2 = cauchy.pdf(aux_domain, *out)

fig_met_df_3.add_trace(go.Scatter(
                 x=aux_domain,
                 y= y2*35,
                 name = 'cauchy_0.9',
                 showlegend = True,
                 line=dict(color='red')
                 ))


fig_met_df_3.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada) (6 pts)

* Implementar función que genere **Lévy Walks** (LW) utilizando pandas.
* Guardar trayectorias en Pandas Data Frame.
* Implementar una función que calcule los valores de **step lenght** de una trayectoria.
* Guardar trayectorias en **pandas** Data Frame.
* Obtener **Step-length** distribution.
* Comparar en gráfica distribución origen vs distribución observada (Histograma).
* Visualizar con plotly.

In [68]:
# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=1.0, beta=1.0, loc=3.0]
Levy_2d_df_1 = levy_flight()#pd.read_csv('drive/MyDrive/Files_colab/levy_6_1.csv')

# Load existing trajectories to test your implementation
# Levy speed = 6
# levy_stable [alpha=0.7, beta=1.0, loc=3.0]
Levy_2d_df_7 = pd.read_csv('drive/MyDrive/Files_colab/levy_6_7.csv')

In [ ]:
# Define your function to compute Step lengths for given trajectory

## start - Add your code here
# def function (trajectory):


    #return sl_df

## end - Add your code here

In [ ]:
# Get Step lengths calling the function
sl_Levy_2d_df_1 = # Add your code here function(Levy_2d_df_1)

# Get Step lengths calling the function
sl_Levy_2d_df_7 = # Add your code here function(Levy_2d_df_7)

In [ ]:
# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here

## end - Add your code here


# Histogram step-length Levy_2d_df_7
## start - Add your code here

## end - Add your code here


# Add origin distributions
## start - Add your code here

## end - Add your code here


fig_met_df_4.show()